In [1]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from keras.preprocessing import image
import numpy as np
#from google.colab.patches import cv2_imshow
import os
import sys
from PIL import Image

Using TensorFlow backend.


In [ ]:
# import the necessary packages
from imutils.video import VideoStream
from imutils import face_utils
import imutils
import time
import dlib
import cv2

In [2]:
# load json and create model
#load the model 
from keras.models import model_from_json
json_file = open('FaceVerification.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("FaceVerification.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
# Compiling and Making New Predictions

# compile loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# initialize dlib's face detector (HOG-based) and then create the
# facial landmark predictor
detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor("C:\\Users\\Allworx User 2.ALLWORXW2\\Anaconda3\\envs\\tf15\\Lib\\site-packages\\face_recognition_models\\models\\shape_predictor_68_face_landmarks.dat")
#predictor = dlib.shape_predictor("C:\\Users\\Allworx User 2.ALLWORXW2\\Anaconda3\\envs\\tf15\\Lib\\site-packages\\face_recognition_models\\models\\shape_predictor_5_face_landmarks.dat")
#predictor = dlib.shape_predictor("C:\\Users\\Allworx User 2.ALLWORXW2\\Anaconda3\\envs\\tf15\\Lib\\site-packages\\face_recognition_models\\models\\mmod_human_face_detector.dat")
# initialize the video stream and sleep for a bit, allowing the


cap = cv2.VideoCapture(0)
cap.set(5, 30)
frameRate = cap.get(5)
print(frameRate)
count=0

# loop over the frames from the video stream
while True:
# grab the frame from the threaded video stream, resize it to
# have a maximum width of 400 pixels, and convert it to
# grayscale
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=800)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale frame
    rects = detector(gray, 0)
    print(len(rects))
    # check to see if a face was detected, and if so, draw the total
# number of faces on the frame
    if len(rects) > 0:
# loop over the face detections
        for rect in rects:
        # compute the bounding box of the face and draw it on the
        # frame
            (bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (bX, bY), (bX + bW, bY + bH), (0, 255, 0), 1)
            text = "{} face(s) found".format(len(rects))
            cv2.putText(frame, text, (bX, bY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw each of them
            for (i, (x, y)) in enumerate(shape):
                cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
                cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

            # show the frame
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF
            # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cv2.destroyAllWindows()
cap.release()

In [4]:
import numpy as np
from keras.preprocessing import image
font = cv2.FONT_HERSHEY_SIMPLEX
#face_cascade = cv2.CascadeClassifier("C:\\Users\\kamma\\Anaconda3\\envs\\vdpnLeP35\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml")
#face_cascade = cv2.CascadeClassifier("C:\\Users\\kamma\\Anaconda3\\envs\\vdpnLeP35\\Library\\etc\\haarcascades\\haarcascade_frontalface_alt_tree.xml")
face_cascade = cv2.CascadeClassifier("C:\\Users\Allworx User 2.ALLWORXW2\\Anaconda3\\envs\\tf15\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml")
#face_cascade = cv2.CascadeClassifier("C:\\Users\\kamma\\Anaconda3\\envs\\vdpnLeP35\\Library\\etc\\haarcascades\\haarcascade_frontalface_alt2.xml")

cap = cv2.VideoCapture(0)
cap.set(5, 30)
frameRate = cap.get(5)
print(frameRate)
count=0
while True:
    #current frame number--this is for frame rate
    count+=1
    frameId=count
    ret, img = cap.read()
    #frameId = cap.get(1)
    #show the mark on face
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    print(frameId)
    print(frameRate)
    if (frameId % (math.floor((frameRate/5))) == 0):
        print("Inside if ")
        for (x,y,w,h) in faces:
            print(faces)
            print("inside for loop")
            print(x,y,w,h)
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 5)  #2 is width of the line 
            #roi_gray = gray[y:y+h, x:x+w]
            #roi_color = img[y:y+h, x:x+w]
            #pridicting with model 
            #converting ndarray to image
            filename ="vijay.jpg"
            crop_img = img[y:y+h, x:x+w]
            cv2.imwrite(filename, crop_img) 
            
            test_image = image.load_img("vijay.jpg", target_size = (64, 64))
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = loaded_model.predict(test_image)          
            #result = loaded_model.predict(img[y:y+h, x:x+w])
            #training_set.class_indices
            if result[0][0] == 1:
                prediction = 'Smile'
            elif result[0][0] == 0 :
                prediction = 'Anger'
            else:
                prediction = 'Nutral'    
            print(prediction)
            #Write text on the rectanguler box.
            cv2.putText(img, prediction, (x, y), font, 0.8, (0, 0, 255), 3, cv2.LINE_AA)     
            cv2.imshow('img',img)
            cv2.imshow('crop_img',crop_img)
    k= cv2.waitKey(5) & 0xff
    if k==27:
        break
        
cv2.destroyAllWindows()
cap.release()

30.00003000003
1
30.00003000003
2
30.00003000003
3
30.00003000003
4
30.00003000003
5
30.00003000003
6
30.00003000003
Inside if 
[[234 138 188 188]]
inside for loop
234 138 188 188
Smile
7
30.00003000003
8
30.00003000003
9
30.00003000003
10
30.00003000003
11
30.00003000003
12
30.00003000003
Inside if 
[[231 128 207 207]]
inside for loop
231 128 207 207
Anger
13
30.00003000003
14
30.00003000003
15
30.00003000003
16
30.00003000003
17
30.00003000003
18
30.00003000003
Inside if 
[[227 131 203 203]]
inside for loop
227 131 203 203
Anger
19
30.00003000003
20
30.00003000003
21
30.00003000003
22
30.00003000003
23
30.00003000003
24
30.00003000003
Inside if 
[[219 132 206 206]]
inside for loop
219 132 206 206
Anger
25
30.00003000003
26
30.00003000003
27
30.00003000003
28
30.00003000003
29
30.00003000003
30
30.00003000003
Inside if 
[[219 131 207 207]]
inside for loop
219 131 207 207
Anger
31
30.00003000003
32
30.00003000003
33
30.00003000003
34
30.00003000003
35
30.00003000003
36
30.00003000003
I

275
30.00003000003
276
30.00003000003
Inside if 
[[205 121 218 218]]
inside for loop
205 121 218 218
Anger
277
30.00003000003
278
30.00003000003
279
30.00003000003
280
30.00003000003
281
30.00003000003
282
30.00003000003
Inside if 
[[203 123 216 216]]
inside for loop
203 123 216 216
Anger
283
30.00003000003
284
30.00003000003
285
30.00003000003
286
30.00003000003
287
30.00003000003
288
30.00003000003
Inside if 
[[201 122 217 217]]
inside for loop
201 122 217 217
Anger
289
30.00003000003
290
30.00003000003
291
30.00003000003
292
30.00003000003
293
30.00003000003
294
30.00003000003
Inside if 
[[206 124 212 212]]
inside for loop
206 124 212 212
Anger
295
30.00003000003
296
30.00003000003
297
30.00003000003
298
30.00003000003
299
30.00003000003
300
30.00003000003
Inside if 
[[204 120 217 217]]
inside for loop
204 120 217 217
Anger
301
30.00003000003
302
30.00003000003
303
30.00003000003
304
30.00003000003
305
30.00003000003
306
30.00003000003
Inside if 
[[204 122 217 217]]
inside for loop


573
30.00003000003
574
30.00003000003
575
30.00003000003
576
30.00003000003
Inside if 
[[238 174 190 190]]
inside for loop
238 174 190 190
Smile
577
30.00003000003
578
30.00003000003
579
30.00003000003
580
30.00003000003
581
30.00003000003
582
30.00003000003
Inside if 
[[203 175 198 198]]
inside for loop
203 175 198 198
Smile
583
30.00003000003
584
30.00003000003
585
30.00003000003
586
30.00003000003
587
30.00003000003
588
30.00003000003
Inside if 
589
30.00003000003
590
30.00003000003
591
30.00003000003
592
30.00003000003
593
30.00003000003
594
30.00003000003
Inside if 
595
30.00003000003
596
30.00003000003
597
30.00003000003
598
30.00003000003
599
30.00003000003
600
30.00003000003
Inside if 
601
30.00003000003
602
30.00003000003
603
30.00003000003
604
30.00003000003
605
30.00003000003
606
30.00003000003
Inside if 
607
30.00003000003
608
30.00003000003
609
30.00003000003
610
30.00003000003
611
30.00003000003
612
30.00003000003
Inside if 
613
30.00003000003
614
30.00003000003
615
30.00

887
30.00003000003
888
30.00003000003
Inside if 
[[247 183 150 150]]
inside for loop
247 183 150 150
Anger
889
30.00003000003
890
30.00003000003
891
30.00003000003
892
30.00003000003
893
30.00003000003
894
30.00003000003
Inside if 
[[251 188 141 141]]
inside for loop
251 188 141 141
Anger
895
30.00003000003
896
30.00003000003
897
30.00003000003
898
30.00003000003
899
30.00003000003
900
30.00003000003
Inside if 
[[250 188 141 141]]
inside for loop
250 188 141 141
Smile
901
30.00003000003
902
30.00003000003
903
30.00003000003
904
30.00003000003
905
30.00003000003
906
30.00003000003
Inside if 
[[251 188 144 144]]
inside for loop
251 188 144 144
Anger
907
30.00003000003
908
30.00003000003
909
30.00003000003
910
30.00003000003
911
30.00003000003
912
30.00003000003
Inside if 
[[250 184 148 148]]
inside for loop
250 184 148 148
Anger
913
30.00003000003
914
30.00003000003
915
30.00003000003
916
30.00003000003
917
30.00003000003
918
30.00003000003
Inside if 
[[250 183 148 148]]
inside for loop


1152
30.00003000003
Inside if 
[[254 196 148 148]]
inside for loop
254 196 148 148
Smile
1153
30.00003000003
1154
30.00003000003
1155
30.00003000003
1156
30.00003000003
1157
30.00003000003
1158
30.00003000003
Inside if 
[[253 195 152 152]]
inside for loop
253 195 152 152
Smile
1159
30.00003000003
1160
30.00003000003
1161
30.00003000003
1162
30.00003000003
1163
30.00003000003
1164
30.00003000003
Inside if 
[[254 196 151 151]]
inside for loop
254 196 151 151
Smile
1165
30.00003000003
1166
30.00003000003
1167
30.00003000003
1168
30.00003000003
1169
30.00003000003
1170
30.00003000003
Inside if 
[[256 197 151 151]]
inside for loop
256 197 151 151
Smile
1171
30.00003000003
1172
30.00003000003
1173
30.00003000003
1174
30.00003000003
1175
30.00003000003
1176
30.00003000003
Inside if 
[[253 193 155 155]]
inside for loop
253 193 155 155
Smile
1177
30.00003000003
1178
30.00003000003
1179
30.00003000003
1180
30.00003000003
1181
30.00003000003
1182
30.00003000003
Inside if 
[[257 201 137 137]]
insi

...

2995
30.00003000003
2996
30.00003000003
2997
30.00003000003
2998
30.00003000003
2999
30.00003000003
3000
30.00003000003
Inside if 
3001
30.00003000003
3002
30.00003000003
3003
30.00003000003
3004
30.00003000003
3005
30.00003000003
3006
30.00003000003
Inside if 
3007
30.00003000003
3008
30.00003000003
3009
30.00003000003
3010
30.00003000003
3011
30.00003000003
3012
30.00003000003
Inside if 
3013
30.00003000003
3014
30.00003000003
3015
30.00003000003
3016
30.00003000003
3017
30.00003000003
3018
30.00003000003
Inside if 
3019
30.00003000003
3020
30.00003000003
3021
30.00003000003
3022
30.00003000003
3023
30.00003000003
3024
30.00003000003
Inside if 
3025
30.00003000003
3026
30.00003000003
3027
30.00003000003
3028
30.00003000003
3029
30.00003000003
3030
30.00003000003
Inside if 
3031
30.00003000003
3032
30.00003000003
3033
30.00003000003
3034
30.00003000003
3035
30.00003000003
3036
30.00003000003
Inside if 
3037
30.00003000003
3038
30.00003000003
3039
30.00003000003
3040
30.00003000003
304

3358
30.00003000003
3359
30.00003000003
3360
30.00003000003
Inside if 
3361
30.00003000003
3362
30.00003000003
3363
30.00003000003
3364
30.00003000003
3365
30.00003000003
3366
30.00003000003
Inside if 
3367
30.00003000003
3368
30.00003000003
3369
30.00003000003
3370
30.00003000003
3371
30.00003000003
3372
30.00003000003
Inside if 
3373
30.00003000003
3374
30.00003000003
3375
30.00003000003
3376
30.00003000003
3377
30.00003000003
3378
30.00003000003
Inside if 
3379
30.00003000003
3380
30.00003000003
3381
30.00003000003
3382
30.00003000003
3383
30.00003000003
3384
30.00003000003
Inside if 
3385
30.00003000003
3386
30.00003000003
3387
30.00003000003
3388
30.00003000003
3389
30.00003000003
3390
30.00003000003
Inside if 
3391
30.00003000003
3392
30.00003000003
3393
30.00003000003
3394
30.00003000003
3395
30.00003000003
3396
30.00003000003
Inside if 
3397
30.00003000003
3398
30.00003000003
3399
30.00003000003
3400
30.00003000003
3401
30.00003000003
3402
30.00003000003
Inside if 
3403
30.0000

3737
30.00003000003
3738
30.00003000003
Inside if 
3739
30.00003000003
3740
30.00003000003
3741
30.00003000003
3742
30.00003000003
3743
30.00003000003
3744
30.00003000003
Inside if 
3745
30.00003000003
3746
30.00003000003
3747
30.00003000003
3748
30.00003000003
3749
30.00003000003
3750
30.00003000003
Inside if 
3751
30.00003000003
3752
30.00003000003
3753
30.00003000003
3754
30.00003000003
3755
30.00003000003
3756
30.00003000003
Inside if 
3757
30.00003000003
3758
30.00003000003
3759
30.00003000003
3760
30.00003000003
3761
30.00003000003
3762
30.00003000003
Inside if 
3763
30.00003000003
3764
30.00003000003
3765
30.00003000003
3766
30.00003000003
3767
30.00003000003
3768
30.00003000003
Inside if 
3769
30.00003000003
3770
30.00003000003
3771
30.00003000003
3772
30.00003000003
3773
30.00003000003
3774
30.00003000003
Inside if 
3775
30.00003000003
3776
30.00003000003
3777
30.00003000003
3778
30.00003000003
3779
30.00003000003
3780
30.00003000003
Inside if 
3781
30.00003000003
3782
30.0000

4056
30.00003000003
Inside if 
[[282 114 229 229]]
inside for loop
282 114 229 229
Anger
4057
30.00003000003
4058
30.00003000003
4059
30.00003000003
4060
30.00003000003
4061
30.00003000003
4062
30.00003000003
Inside if 
[[286 120 219 219]]
inside for loop
286 120 219 219
Anger
4063
30.00003000003
4064
30.00003000003
4065
30.00003000003
4066
30.00003000003
4067
30.00003000003
4068
30.00003000003
Inside if 
[[286 120 220 220]]
inside for loop
286 120 220 220
Anger
4069
30.00003000003
4070
30.00003000003
4071
30.00003000003
4072
30.00003000003
4073
30.00003000003
4074
30.00003000003
Inside if 
[[286 121 221 221]]
inside for loop
286 121 221 221
Anger
4075
30.00003000003
4076
30.00003000003
4077
30.00003000003
4078
30.00003000003
4079
30.00003000003
4080
30.00003000003
Inside if 
[[298 126 211 211]]
inside for loop
298 126 211 211
Anger
4081
30.00003000003
4082
30.00003000003
4083
30.00003000003
4084
30.00003000003
4085
30.00003000003
4086
30.00003000003
Inside if 
[[299 123 213 213]]
insi

4320
30.00003000003
Inside if 
[[270 122 217 217]]
inside for loop
270 122 217 217
Anger
4321
30.00003000003
4322
30.00003000003
